# Network Representation for Use-Cases
Author Ilias Parmaksizoglou

This notebook provided a quick tour on how to use the Network Representation for Use-Cases: The needed input is: 
* O-D matrix of randomized trips within the examined area
* OSM map of examined area in .graphml format


In [122]:
import osmnx as ox
import networkx as nx
import random 
import igraph as ig
import os
import time as tm
import matplotlib.pyplot as plt

In [123]:
# Initialize globals
globals()["num_trips"] = 1000
globals()["timesteps"] = 10
globals()["period"] = 60
globals()["weight"] = "travel_time"

In [124]:
def transform_nx_to_igragh(G,flag=True):
    osmids = list(G.nodes)
    if flag:
        G = nx.relabel.convert_node_labels_to_integers(G)
    G_ig = ig.Graph(directed=True)
    G_ig.add_vertices(G.nodes)
    G_ig.add_edges(G.edges())
    G_ig.vs["osmid"] = osmids
    G_ig.es[weight] = list(nx.get_edge_attributes(G, weight).values())
    osmid_values = {k: v for k, v in zip(G.nodes, osmids)}
    nx.set_node_attributes(G, osmid_values, "osmid")
    assert len(G.nodes()) == G_ig.vcount()
    assert len(G.edges()) == G_ig.ecount()
    return G,G_ig
    

In [125]:
def adjust_for_next_period(dict_path,routes,initial_start):
    for args in dict_path.items():
        idx = args[0][2]
        if args[1][1] >= (step)*period and args[1][0]< (step)*period:
            origin = args[0][1]
            initial_start[idx] = args[1][1] # Fix re-computing to be executed only once
            index = routes[idx].index(origin)
            routes[idx] = routes[idx][index:]
        elif args[0][1] == routes[idx][-1] and args[1][1] <= (step)*period:
            routes[idx] = []
            initial_start[idx] = -1
    return routes,initial_start

In [126]:
def generate_random_trips(G,G_ig,step,routes,initial_start):
    random.seed(step)
    for t in range(0,num_trips):
        route = None
        while route == None:
            x = random.randrange(0,len(list(G.nodes)))          
            y = random.randrange(0,len(list(G.nodes)))
            route = G_ig.get_shortest_paths(v=list(G.nodes())[x], to=list(G.nodes())[y], weights=weight)[0]
        idx_t = step*num_trips + t # Identifier of trip
        initial_start[idx_t] = step*period # Original Start time of trip
        routes.append(route)
    return routes,initial_start

In [127]:
def update_dicts(G,routes,initial_start,step):
    dict_path = {}
    edges_dict_veh = {}
    for idx_r,route in enumerate(routes):
        cur_time = initial_start[idx_r]
        for v in range(len(route)-1):
            duration = G.edges[route[v], route[v+1],0]['travel_time']
            dict_path[(route[v],route[v+1],idx_r)] = (round(cur_time,2) ,round(cur_time+duration,2))

            if  (route[v],route[v+1],step) not in edges_dict_veh.keys(): 
                edges_dict_veh[((route[v],route[v+1]),step)] = 0

            if (step)*period <= dict_path[(route[v],route[v+1],idx_r)][0] and (step+1)*period > dict_path[(route[v],route[v+1],idx_r)][0]:
                edges_dict_veh[((route[v],route[v+1]),step)]+=1
            elif (step)*period >= dict_path[(route[v],route[v+1],idx_r)][0] and (step)*period < dict_path[(route[v],route[v+1],idx_r)][1]:
                edges_dict_veh[((route[v],route[v+1]),step)]+=1

            cur_time+=duration
    return dict_path,edges_dict_veh

In [128]:
def update_gdfs(gdf_edges,edges_dict_veh,step,length,max_speed):

    # Update speeds of edges
    edges_speed = {}
    edges_time = {}
    edges_drop = {}
    for index, row in gdf_edges.iterrows():
    
        if ((index[0],index[1]),step) in edges_dict_veh.keys():
            dens = (edges_dict_veh[((index[0],index[1]),step)])/length[index]
        else:
            dens = 0

        if type(row["lanes"]) == list :
            lanes = int(row["lanes"][0])
        else:
            lanes = int(row["lanes"])

        # Travel Speed Update (km per hour)
        sp = max(0.01,max_speed[index]*(1-(dens/(0.15*lanes))))

        # Travel time update (seconds)
        new_travel_time = 3600*(0.001*length[index])/sp
        edges_speed[index] = sp
        edges_time[index] = new_travel_time
        edges_drop[index] = 1-sp/max_speed[index]

    gdf_edges['speed_kph'] = gdf_edges.index.to_series().map(edges_speed)
    gdf_edges['travel_time'] = gdf_edges.index.to_series().map(edges_time)
    gdf_edges['drop'] = gdf_edges.index.to_series().map(edges_drop)
    return gdf_edges

In [129]:
# Loading OSM map from file
core_dir = os.path.join(os.getcwd(),"core\maps")
area = "delft.graphml"
G = ox.load_graphml(os.path.join(core_dir,area))

# Break down to gdf_nodes and gdf_edges
G,G_ig = transform_nx_to_igragh(G)
gdf_nodes,gdf_edges = ox.graph_to_gdfs(G, nodes=True, edges=True, node_geometry=True, fill_edge_geometry=True)

# Store Max Speeds and Lengths
max_speed = {}
length = {}
for index, row in gdf_edges.iterrows():
    max_speed[index] = int(row["speed_kph"])
    length[index] = (row["length"])
            

In [ ]:
# Initialize Dictionaries and Lists
routes = [] # List of Networkx routes in the map (Vertex-to-Vertex)
initial_start = {}
edges_speed = {}
edges_time = {}

for step in range(timesteps):

    # Step 0 - Adjust route based on next time step
    step0 = tm.time()
    if step == 0:
        pass
    else:
        routes,initial_start = adjust_for_next_period(dict_path,routes,initial_start)
    step1 = tm.time()
    #print("step0:", step1-step0)

    # Step 1 - Generate new routes
    routes,initial_start = generate_random_trips(G,G_ig,step,routes,initial_start)
    step2 = tm.time()
    #print("step1:",step2-step1) 
    
    # Step 2 - Update dict_path edges and number of vehicles
    dict_path,edges_dict_veh = update_dicts(G,routes,initial_start,step)
    step2 = tm.time()
    #print("step2:",step2-step1) 

    # Step 3 - Update gdf_edges 
    gdf_edges = update_gdfs(gdf_edges,edges_dict_veh,step,length,max_speed)
    step3 = tm.time()
    #print("step3:",step3-step2)

    # Step 4 - Restructure Graph
    G = ox.graph_from_gdfs(gdf_nodes, gdf_edges)
    G,G_ig = transform_nx_to_igragh(G,flag=False)
    gdf_nodes,gdf_edges = ox.graph_to_gdfs(G, nodes=True, edges=True, node_geometry=True, fill_edge_geometry=True)
    step4 = tm.time()
    #print("step4:",step4-step3) 

    # Step 5 - Draw network status
    fig_heatmap1 = plt.figure(figsize=(20,16))
    ax_heatmap1 = plt.axes()
    ax_heatmap1.set(facecolor = "white")
    ax_heatmap1.set_title("Speed Drop Heatmap in Delft Area step = {}".format(step), fontsize=20)
    gdf_edges.plot(ax=ax_heatmap1, cmap="viridis", column="drop", legend=True)
    core_dir = os.path.join(os.getcwd(),"core\pics")
    plt.savefig(f"{core_dir}\step{step}")
    step5 = tm.time()
    #print("step5:",step5-step4) 

In [ ]:
import os
import imageio

cwd = os.getcwd()
png_dir = 'core\pics'
images = []
for file_name in sorted(os.listdir(os.path.join(cwd,png_dir))):
    if file_name.endswith('.png'):
        file_path = os.path.join(png_dir, file_name)
        images.append(imageio.imread(file_path))
imageio.mimsave(os.path.join(cwd,png_dir,'movie.gif'), images,fps = 2)